In [18]:
import requests
import pandas as pd
import plotly          
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [19]:
#Create Age index
data = {'Age': [], 'AgeD': []}
for year in range(18,40):
    for month in range(0, 12):
        data['Age'].append(str(year) + " years " + str(month) + " months")
        data['AgeD'].append(365.25*year + 30.5*month)

In [20]:
birthdate = "10/24/85"
def addGoals(name, birthdate, url):
    
    headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    pageTree = requests.get(url, headers=headers)
    df = pd.read_html(pageTree.text)
    df = df[2]  #Main stats table with goals and goals date
    
    check = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9', "H", "A"} #Valid Entry Check (valid goal scored & Home/Away venue)
    data[name] = []
    curr = 0 
    goals = 0
    birthdate = datetime.strptime(birthdate, '%m/%d/%y')
    
    for i in df.index:
        if df.loc[i]['Venue'] not in check: #If not valid entry skip iteration
            continue
        age = datetime.strptime(df.loc[i]['Date'], '%m/%d/%y') - birthdate
        age = age.days
        
        if df.loc[i]['Unnamed: 9'] in check:
            goals += int(df.loc[i]['Unnamed: 9'])
            
        if age >= data['AgeD'][curr]:
            curr += 1
            data[name].append(goals)
            
    curr += 1
    while curr <= len(data["Age"]): #Fill empty indexes
        data[name].append(goals)
        curr += 1

#addGoals("Wayne Rooney", birthdate, "https://www.transfermarkt.us/wayne-rooney/leistungsdatendetails/spieler/3332/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id=")            

In [21]:
player_index = {
    "Wayne Rooney": ['10/24/85', 'https://www.transfermarkt.us/wayne-rooney/leistungsdatendetails/spieler/3332/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id='],
    "Michael Owen": ['12/14/79', 'https://www.transfermarkt.us/michael-owen/leistungsdatendetails/spieler/1397/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id='],
    "Alan Shearer": ['8/13/70', 'https://www.transfermarkt.us/alan-shearer/leistungsdatendetails/spieler/3110/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id='],
    "Frank Lampard": ['6/20/78', 'https://www.transfermarkt.us/frank-lampard/leistungsdatendetails/spieler/3163/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id='],
    "Sergio Aguero": ['6/2/88', 'https://www.transfermarkt.us/sergio-aguero/leistungsdatendetails/spieler/26399/plus/0?saison=&verein=&liga=&wettbewerb=GB1&pos=&trainer_id=']
}

for name in player_index:
    addGoals(name, player_index[name][0], player_index[name][1]) #(name, bday, url)

In [22]:
#df1 = pd.DataFrame(data, index = data['Age']) 
#df1.to_excel("PremiereLeagueGoalsByAge.xlsx")
#keys = [i for i in range(len(data['Age']))]
del data['AgeD']
Data = pd.DataFrame(data) 
Data

,Age,Wayne Rooney,Michael Owen,Alan Shearer,Frank Lampard,Sergio Aguero
0,18 years 0 months,7,6,2,0,2
1,18 years 1 months,7,7,3,0,2
2,18 years 2 months,9,13,3,0,3
3,18 years 3 months,10,15,3,0,6
4,18 years 4 months,12,17,4,0,8
...,...,...,...,...,...,...
259,39 years 7 months,208,150,260,177,184
260,39 years 8 months,208,150,260,177,184
261,39 years 9 months,208,150,260,177,184
262,39 years 10 months,208,150,260,177,184


In [25]:
"Create Frames for Plotly Animation"
n_frame = dict()
x = 0
color = {"Wayne Rooney": '#4287f5', "Michael Owen": '#15edac', "Alan Shearer": '#f70f45', "Frank Lampard": '#f70f23', "Sergio Aguero": "#4287f2"}
while x < len(Data['Age']):
    dataFrame = Data[(Data['Age']==Data['Age'][x])]
    currFrame = []
    for i in dataFrame:
        if i == 'Age':
            continue
        currFrame.append([i, dataFrame['Age'].to_string(index=False), dataFrame[i].to_string(index=False), color[i]])

    currFrame = pd.DataFrame(currFrame, columns = ['Name', 'Age', 'Goals', 'color_code'])
    currFrame = currFrame.sort_values(by=['Goals'])
    n_frame[x] = currFrame
    x += 1

n_frame

{0:             Name                Age Goals color_code
 3  Frank Lampard  18 years 0 months     0    #f70f23
 2   Alan Shearer  18 years 0 months     2    #f70f45
 4  Sergio Aguero  18 years 0 months     2    #4287f2
 1   Michael Owen  18 years 0 months     6    #15edac
 0   Wayne Rooney  18 years 0 months     7    #4287f5,
 1:             Name                Age Goals color_code
 3  Frank Lampard  18 years 1 months     0    #f70f23
 4  Sergio Aguero  18 years 1 months     2    #4287f2
 2   Alan Shearer  18 years 1 months     3    #f70f45
 0   Wayne Rooney  18 years 1 months     7    #4287f5
 1   Michael Owen  18 years 1 months     7    #15edac,
 2:             Name                Age Goals color_code
 3  Frank Lampard  18 years 2 months     0    #f70f23
 1   Michael Owen  18 years 2 months    13    #15edac
 2   Alan Shearer  18 years 2 months     3    #f70f45
 4  Sergio Aguero  18 years 2 months     3    #4287f2
 0   Wayne Rooney  18 years 2 months     9    #4287f5,
 3:             

In [26]:
#Plotly figure
fig = go.Figure(
    data=[
        go.Bar(
        x=n_frame[1]['Goals'], y=n_frame[1]['Name'],orientation='h',
        text=n_frame[1]['Goals'], texttemplate='%{text:.3s}',
        textfont={'size':18}, textposition='inside', insidetextanchor='middle',
        width=0.9, marker={'color':n_frame[1]['color_code']})
    ],
    layout=go.Layout(
        xaxis=dict(autorange=True, title=dict(text='Goals',font=dict(size=18))),
        yaxis=dict(autorange=True, tickfont=dict(size=14)),
        title=dict(text='Premiere League Goals by Age: 18 years 0 months ',font=dict(size=28),x=0.5,xanchor='center'),
        # Add button
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None,
                          {"frame": {"duration": 1000, "redraw": True},
                          "transition": {"duration":250,
                          "easing": "linear"}}]
            )]
        )]
    ),
    frames=[
            go.Frame(
                data=[
                        go.Bar(x=value['Goals'], y=value['Name'],
                        orientation='h',text=value['Goals'],
                        marker={'color':value['color_code']})
                    ],
                layout=go.Layout(
                        xaxis=dict(autorange=True, title=dict(text='Goals',font=dict(size=18))),
                        yaxis=dict(autorange=True,tickfont=dict(size=14)),
                        title=dict(text='Premiere League Goals by Age: '+str(value['Age'].values[0]),
                        font=dict(size=28))
                    )
            )
        for key, value in n_frame.items()
    ]
)

In [27]:
fig.write_html("Plotly.html") #Export

In [28]:
import bar_chart_race as bcr

In [29]:
Data = pd.DataFrame(data, index = data["Age"])
Data = Data.drop('Age', 1)
bcr.bar_chart_race(df = Data, title = "Race", n_bars=3)

c:\users\taha4\appdata\local\programs\python\python38-32\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning:

FixedFormatter should only be used together with FixedLocator

c:\users\taha4\appdata\local\programs\python\python38-32\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning:

FixedFormatter should only be used together with FixedLocator

